<h1>Final Year Project - The Third Eye </h1>
Done by: See Zhuo Yi Joey (2011927), Liu Zhen (2021250), Koh Hui Lyn (2021672) and Ang Jun Hoa (2040295)
<br/>Project Aim: Using computer vision to aid coach’s analysis of a bowler’s performance by producing consistent and accurate intelligent analysis.
<br/>Modules Required: MediaPipe, OpenCV


<h2>Installing Required Packages</h2>

In [1]:
# pip install mediapipe
# pip install opencv-python

<h2>Importing</h2>

In [2]:
import cv2
import math as m
import mediapipe as mp

<h2>Functions</h2>

In [3]:
# Find Distance between 2 points
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

# Calculate angle
def findAngle(x1, y1, x2, y2):
    theta = m.acos((y2 -y1)*(-y1) / (m.sqrt((x2 - x1)**2 + (y2 - y1)**2) * y1))
    degree = int(180/m.pi)*theta
    return degree


<h2>Utils</h2>

In [4]:
# Font (For OpenCV Video)
font = cv2.FONT_HERSHEY_SIMPLEX

# Colors 
blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)

In [5]:
# Using Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Choose which video to use
# ((For webcam input replace file name with 0))
file_name = './videos/moving1.mov'
cap = cv2.VideoCapture(file_name)

# CV2  properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Output video details
video_output = cv2.VideoWriter('output.mp4', fourcc, fps, frame_size)

<h2>Main Code</h2>

In [6]:
print('Starting...')
steps = 0
stage = None
while cap.isOpened():
    # Capture frames
    success, image = cap.read() 
    if not success:
        print("No frames left to process!!!")
        break
    # Get fps, height and width
    fps = cap.get(cv2.CAP_PROP_FPS)
    h, w = image.shape[:2]
    # Convert the BGR image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Process the frame with Mediapipe Pose
    keypoints = pose.process(image)
    # Convert the image back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    #============ Getting landmarks ============
    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    # Left shoulder
    l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
    l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
    # Right shoulder
    r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
    r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
    # Left ear
    l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
    l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
    # Left hip
    l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
    l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)

    # Ankles for feet distance calculation
    l_ank_x = int(lm.landmark[lmPose.LEFT_ANKLE].x * w)
    l_ank_y = int(lm.landmark[lmPose.LEFT_ANKLE].y * h)
    r_ank_x = int(lm.landmark[lmPose.RIGHT_ANKLE].x * w)
    r_ank_y = int(lm.landmark[lmPose.RIGHT_ANKLE].y * h)

    #============ Functions ============
    # Check for Camera Alignment to be in Proper Sideview
    offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
    if offset < 100:
        cv2.putText(image, str(int(offset)) + ' Aligned', (w - 150, 30), font, 0.9, green, 2)
    else:
        cv2.putText(image, str(int(offset)) + ' Not Aligned', (w - 150, 30), font, 0.9, red, 2)

    feetDist = findDistance(l_ank_x, l_ank_y, r_ank_x, r_ank_y)

    # Steps Counter (To be improved - ie Thresholds improvements)
    if steps < 5:
        if feetDist > 145 and stage == 'up':
            steps += 1
            stage = "down"
        elif feetDist < 130:
            stage = "up"
    
    # Calculate torso and neck angles
    neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
    torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)

    #============ Annotations onto video ============
    # Draw landmarks
    cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
    cv2.circle(image, (l_ear_x, l_ear_y), 7, yellow, -1)
    cv2.circle(image, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
    # Right shoulder is pink ball
    cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)
    cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)
    cv2.circle(image, (l_hip_x, l_hip_y - 100), 7, yellow, -1)

    # Text for Neck/Torso Angle, Feet distance & Steps
    angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination)) + ' Feet distance: '+ str(int(feetDist)) + ' Steps: '+ str(int(steps))
    cv2.putText(image, angle_text_string, (10, 50), font, 1.6, dark_blue, 4)

    # Display angles on the annotation
    cv2.putText(image, str(int(neck_inclination)), (l_shldr_x + 10, l_shldr_y), font, 1.2, pink, 2)
    cv2.putText(image, str(int(torso_inclination)), (l_hip_x + 10, l_hip_y), font, 1.2, pink, 2)

    # Join landmarks
    cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), green, 4)
    cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), green, 4)
    cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, 4)
    cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, 4)
    

    # Write frames.
    video_output.write(image)
print('Video is done!')
cap.release()
video_output.release()

Starting...
No frames left to process!!!
Video is done!
